# NeuMF para recomendação de receitas de cozinha

Esse projeto consiste no treinamento do modelo NeuMF na base de dados de food.com e a comparação do RMSE com outros modelos de filtragem colaborativa

## Fatoração de Matrizes

Os modelos a comparar são

## Funk SVD

$$\hat{r}_{ui} = \mu + b_u + b_i + p_u^T\cdot q_i$$

In [ ]:
def fit_svd(
    train_data, n_users, n_items, k, α1=.01, α2=.01, α3=.01, α4=.01, λ1=.01, λ2=.01, n_iters=20
):
    bu = np.zeros(n_users, np.double)
    bi = np.zeros(n_items, np.double)  
    P = np.random.normal(0, .1, (n_users, k))
    Q = np.random.normal(0, .1, (n_items, k))
    μ = np.mean(train_data[:, 2])
    
    for u, i, r in train_data:
        pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u])
        error = r - pred

        # Updating
        bu[u] += α1 * (error - λ1*bu[u])
        bi[i] += α2 * (error - λ1*bi[i])

        Pu = P[u]
        Qi = Q[i]
        P[u] += α3*(error*Qi - λ2*Pu)
        Q[i] += α4*(error*Pu - λ2*Qi)
    
    return μ, bu, bi, P, Q

## SVD++

$$\hat{r}_{ui} = b_{ui} + q_i^T\cdot \left(p_u + |N(u)|^{-\frac{1}{2}} \sum_{j\in N(u)} y_j\right)$$

* $N(u)$: conjunto de itens avaliados pelo usuário $u$
* $Y$ matriz de itens $\times$ fatores.

In [ ]:
    for u, i, r in train_data:
        Ru = Rus[u]
        sqrt_Ru = np.sqrt(len(Ru))

        implicit_feedback = np.zeros(k)
        for j in Ru:
            implicit_feedback += Y[j]
        implicit_feedback /= (sqrt_Ru+1e-15)

        pred = μ + bu[u] + bi[i] + np.dot(Q[i], P[u] + implicit_feedback)
        error = r - pred

        # Updating
        bu[u] += α1 * (error - λ1*bu[u])
        bi[i] += α2 * (error - λ1*bi[i])

        Pu = P[u]
        Qi = Q[i]
        P[u] += α3*(error*Qi - λ2*Pu)
        Q[i] += α4*(error*(Pu+implicit_feedback) - λ2*Qi)

        term_1 = error*(Qi/(sqrt_Ru+1e-15))
        for j in Ru:
            Y[j] += α5*(term_1 - λ1*Y[j])

## gSVD++

Usando tags como gêneros.

$$\hat{r}_{ui} = b_{ui} + \left(q_i+|G(i)|^{-\alpha}\sum_{g\in G(i)} x_g\right)^T\left(p_u + |N(u)|^{-\frac{1}{2}}\sum_{j\in N(u)}y_j\right)$$

* $G(i)$: gêneros aos que pertence o item $i$.
* $\alpha=1$
* $X$ matriz de gêneros $\times$ fatores.

In [ ]:
    for u, i, r in train_data:
        Ru = Rus[u]
        sqrt_Ru = np.sqrt(len(Ru))

        implicit_feedback = np.zeros(k)
        for j in Ru:
            implicit_feedback += Y[j]
        implicit_feedback /= (sqrt_Ru+1e-15)
        
        Gi = Gis[i]
        genres_feedback = np.zeros(k)
        for g in Gi:
            genres_feedback += X[g]
        genres_feedback /= len(Gi)

        pred = μ + bu[u] + bi[i] + np.dot(Q[i] + genres_feedback, P[u] + implicit_feedback)
        error = r - pred

In [ ]:
        # Updating
        bu[u] += α1 * (error - λ1*bu[u])
        bi[i] += α2 * (error - λ2*bi[i])

        Pu = P[u]
        Qi = Q[i]
        P[u] += α3*(error*Qi - λ3*Pu)
        Q[i] += α4*(error*(Pu+implicit_feedback) - λ4*Qi)
    
        term_1 = error*(1/len(Gi))*(Pu + implicit_feedback)
        for g in Gi:
            X[g] += α6*(term_1 - λ5*X[g])
            
        term_2 = error*(1/sqrt_Ru)*(Qi + genres_feedback)
        for j in Ru:
            Y[j] += α5*(term_2 - λ6*Y[j])

## Neural Collaborative Filtering

* Predição si o item tem probabilidade de ser de interesse para o usuário.
* Combina a fatoração de matrizes com uma rede neuronal fully connected.
* Composto por dois ramos:
    * General Matrix Factorization
    * Multi-layer Perceptron

* A entrada desse modelo são dos vetores $v_u$ com $N$ elementos e $v_i$ com $M$ elementos
* Esses vetores estão no formato one-hot encoded

### General Matrix Factorization

* Define as matrizes
    * $P^{GMF}$ de $N\times k_{gmf}$
    * $Q^{GMF}$ de $M\times k_{gmf}$
    * $h$ de $k_{gmf}\times 1$

A função de predição e dada por:

$$
\begin{align}
    \phi^{GMF} &= P^{GMF^T}\cdot v_u \odot Q^{GMF^T}\cdot v_i\\
    \hat{y}_{ui} &= f(h^T\cdot (\phi^{GMF}))
\end{align}
$$

![gmf](https://i.pinimg.com/564x/8b/a8/f5/8ba8f5f7b032859983433a126baa9d4e.jpg)

### Multi-Layer Perceptron

* Aplica uma rede neuronal sobre a concatenação dos vetores $v_u$ e $v_i$.
* A ativação da última capa na rede neuronal usada no paper e a sigmoide.

A função de predição e dada por

$$
\begin{align}
    \phi^{MLP} &= mlp\left(\begin{bmatrix}P^{MLP^T}\cdot v_u\\Q^{MLP^T}\cdot v_i\end{bmatrix}\right)\\
    \hat{y}_{ui} &= \sigma\left(h^T\cdot\phi^{MLP}\right)
\end{align}
$$

### NeuMF

Combinando os modelos

$$
\begin{align}
    \phi^{GMF} &= P^{GMF^T}\cdot v_u \odot Q^{GMF^T}\cdot v_i\\
    \phi^{MLP} &= mlp\left(\begin{bmatrix}P^{MLP^T}\cdot v_u\\Q^{MLP^T}\cdot v_i\end{bmatrix}\right)\\
    \hat{y}_{ui} &= \sigma\left(h^T\cdot\begin{bmatrix}\phi^{GMF}\\\phi^{MLP}\end{bmatrix}\right)
\end{align}
$$

![neumf](https://nipunbatra.github.io/blog/images/neumf.png)

In [ ]:
class GMF(nn.Module):
    def __init__(self, n_users, n_items, k):
        super(GMF, self).__init__()

        self.P = nn.Embedding(n_users, k)
        self.Q = nn.Embedding(n_items, k)
        self.h = nn.Linear(k, 1, bias=False)

        nn.init.normal_(self.P.weight, std=0.01)
        nn.init.normal_(self.Q.weight, std=0.01)
        nn.init.normal_(self.h.weight, std=0.01)

    def forward(self, user_ids, item_ids):
        pu = self.P(user_ids)
        qi = self.Q(item_ids)
        X = pu * qi
        return X

In [ ]:
class MLP(nn.Module):
    def __init__(self, n_users, n_items, k, layer_sizes):
        super(MLP, self).__init__()

        self.P = nn.Embedding(n_users, k)
        self.Q = nn.Embedding(n_items, k)

        self.layers = nn.ModuleList()
        prev_size = 2 * k
        for size in layer_sizes:
            self.layers.append(nn.Linear(prev_size, size))
            prev_size = size
            
        nn.init.normal_(self.P.weight, std=0.01)
        nn.init.normal_(self.Q.weight, std=0.01)
        for layer in self.layers:
            nn.init.normal_(layer.weight, std=0.01)

    def forward(self, user_ids, item_ids):
        pu = self.P(user_ids)
        qi = self.Q(item_ids)

        X = torch.concat([pu, qi], dim=1)
        for layer in self.layers:
            X = F.relu(layer(X))

        return X

In [ ]:
class NeuMF(nn.Module):
    def __init__(
            self, gmf: GMF, mlp: MLP, mlp_out_size: int, gmf_out_size: int
    ):
        super(NeuMF, self).__init__()

        self.gmf: GMF = deepcopy(gmf)
        self.mlp: MLP = deepcopy(mlp)

        self.h = nn.Linear(mlp_out_size + gmf_out_size, 1)
        nn.init.normal_(self.h.weight, std=0.01)

    def forward(self, user_ids, item_ids):
        Xg: GMF = self.gmf(user_ids, item_ids)
        Xm: MLP = self.mlp(user_ids, item_ids)

        out = self.h(torch.concat([Xg, Xm], dim=1))
        return out

## Avaliação

Para este projeto se escolheu o modelo NeuMF pela capacidade de aprendizado de projeções não lineares e para analisar como e seu desempenho comparado aos modelos clássicos

O dataset fui obtido em [kaggle](https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions)

### Pre processamento

* Re definição dos ids de usuários e itens porque originalmente não eram contíguos.

* Foram combinados os ingredientes e as instruções de preparação num só texto e foi aplicado TF-IDF.

* Foi construída uma matriz esparsa de itens $\times$ tags, com valores 1 somente nas colunas dos tags aos que pertence cada receita.

* Se criou um novo csv para as receitas com os novos ids dos itens, minutos de preparação, informação nutricional, número de passos e ingredientes.

* Se criou um novo csv das interações com os novos ids dos usuários e os itens.


* Foi feito um split de 70% dos dados para treinamento, 20% para validação e 10% para teste.

* O treinamento se fez no conjunto de teste e se usou o conjunto de validação para controlar a capacidade de generalização do modelo

* Se treinaram os modelos SVD, SVD++, gSVD++ (usando os tags como gêneros) e NeuMF

* Adicional foi treinada uma modificação do NeuMF com dois novos ramos
    * Um mlp sobre um vetor com os minutos de preparação, informação nutricional, número de passos e número de ingredientes
    * Outro mlp sobre o vetor dos tags + o vetor tfidf da preparação.

## Resultados

### SVD

1536 combinações de parâmetros

Melhores parâmetros

$k=4$, $\alpha_1=0.005$, $\alpha_2=0.005$, $\alpha_3=0.005$, $\alpha_4=0.005$, $\lambda_1=0.01$, $\lambda_2=0.1$

### SVD++

20 combinações de parâmetros

Melhores parâmetros

$k=4$, $\alpha_1=0.005$, $\alpha_2=0.005$, $\alpha_3=0.005$, $\alpha_4=0.005$, $\alpha_5=0.005$, $\lambda_1=0.1$, $\lambda_2=0.1$

### gSVD++

15 combinações de parâmetros

Melhores parâmetros

$k=5$

$\alpha_1=0.006$, $\alpha_2=0.006$, $\alpha_3=0.005$, $\alpha_4=0.005$, $\alpha_5=0.005$, $\alpha_6=0.006$, 

$\lambda_1=0.01$, $\lambda_2=0.1$, $\lambda_3=0.01$, $\lambda_4=0.1$, $\lambda_5=0.01$, $\lambda_6=0.1$

### NeuMF

5 combinações de parâmetros

Melhores parâmetros

$k_{gmf}=8$

$k_{mlp}=8$

$layers=[16, 32, 16, 8]$

$lr=0.001$

### NeuMF modificado

5 combinações de parâmetros

Melhores parâmetros

$k_{gmf}=8$

$k_{mlp}=8$

$layers=[32, 16, 8]$

$feature\_layers=[16, 8]$

$text\_layers=[128, 64, 32, 16, 8]$

$lr=0.001$

## Resultados

|   Modelo  | RMSE Validação | RMSE Teste |
|:---------:|:--------------:|:----------:|
|    SVD    |    0.898688    |  0.896018  |
|   SVD++   |    0.898898    |  0.896148  |
|   gSVD++  |    0.898860    |  0.896319  |
|   NeuMF   |    0.904851    |  0.905242  |
| NeuMF ext |    0.915856    |  0.916087  |

## Conclusões

* O resultado esperado e que os modelos mais complexos tenham um menor erro no conjunto de teste.
* Os resultados dos experimentos nos mostram que o modelo mais simples foi o melhor.

Possiveis razoes

* Pode ser que os melhores parâmetros não foram encontrados pelo tempo que demorou treinar o SVD++ e gSVD++.
* NeuMF: pode ser muito complexo para a estrutura dos dados, resultando em overfit.
* NeuMF ext: e possível que so adicionar informação sobre os itens não seja suficiente.

## Referências

* Sheldon Jay Axler. Linear Algebra Done Right. Undergraduate Texts in Mathematics. Springer, New York, 1997.
* Simon Funk. Netflix update: Try this at home. https://sifter.org/simon/journal/20061211.html, 2006.
* Xiangnan He, Lizi Liao, Hanwang Zhang, Liqiang Nie, Xia Hu, and Tat-Seng Chua. Neural collaborative filtering. In Proceedings of the 26th International Conference on World Wide Web, WWW ’17, page 173–182, Republic and Canton of Geneva, CHE, 2017. International World Wide Web Conferences Steering Committee.
* Yehuda Koren. Factorization meets the neighborhood: A multifaceted collaborative filtering model. In Proceedings of the 14th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, KDD ’08, page 426–434, New York, NY, USA, 2008. Association for Computing Machinery.
* Marcelo Garcia Manzato. Gsvd++: Supporting implicit feedback on recommender systems with metadata awareness. In Proceedings of the 28th Annual ACM Symposium on Applied Computing, SAC ’13, page 908–913, New York, NY, USA, 2013. Association for Computing Machinery.

Repositório de Github

https://github.com/nubol23/SCC5966-recommender-systems/tree/master/ProjectFood